In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [28]:
round1_1 = pd.read_csv('data/trades_round_1_day_-2_wn.csv', sep=';')
round1_2 = pd.read_csv('data/trades_round_1_day_-1_wn.csv', sep=';')
round1_3 = pd.read_csv('data/trades_round_1_day_0_wn.csv', sep=';')

round3_1 = pd.read_csv('data/trades_round_3_day_0_wn.csv', sep=';')
round3_2 = pd.read_csv('data/trades_round_3_day_1_wn.csv', sep=';')
round3_3 = pd.read_csv('data/trades_round_3_day_2_wn.csv', sep=';')

round4_1 = pd.read_csv('data/trades_round_4_day_1_wn.csv', sep=';')
round4_2 = pd.read_csv('data/trades_round_4_day_2_wn.csv', sep=';')
round4_3 = pd.read_csv('data/trades_round_4_day_3_wn.csv', sep=';')

SEASHELLS = "SEASHELLS"
AMETHYSTS = "AMETHYSTS"
STARFRUIT = "STARFRUIT"
ORCHIDS = "ORCHIDS"
CHOCOLATE = "CHOCOLATE"
STRAWBERRIES = "STRAWBERRIES"
ROSES = "ROSES"
GIFT_BASKET = "GIFT_BASKET"
COCONUT = "COCONUT"
COCONUT_COUPON = "COCONUT_COUPON"

r1_products = [AMETHYSTS, STARFRUIT]
r3_products = [CHOCOLATE, STRAWBERRIES, ROSES, GIFT_BASKET]
r4_products = [COCONUT, COCONUT_COUPON]

In [24]:
round_1 = pd.concat([round1_1, round1_2, round1_3])
round_3 = pd.concat([round3_1, round3_2, round3_3])
round_4 = pd.concat([round4_1, round4_2, round4_3])

r1_traders = round_1["buyer"].unique()
r3_traders = round_3["buyer"].unique()
r4_traders = round_4["buyer"].unique()

In [29]:
# Trader PnLs

def calc_pnl(trader, trades):
    buys = trades.apply(lambda row: -row["price"] * row["quantity"] if row["buyer"] == trader else 0, axis=1)
    sells = trades.apply(lambda row: row["price"] * row["quantity"] if row["seller"] == trader else 0, axis=1)
    pnl = buys.sum() + sells.sum()
    return pnl

r1_pnls = {trader: {} for trader in r1_traders}
r3_pnls = {trader: {} for trader in r3_traders}
r4_pnls = {trader: {} for trader in r4_traders}


for trader in r1_traders:
    data = {product: round_1[round_1["symbol"] == product] for product in r1_products}
    for product, trades in data.items():
        r1_pnls[trader][product] = calc_pnl(trader, trades)

for trader in r3_traders:
    data = {product: round_3[round_3["symbol"] == product] for product in r3_products}
    for product, trades in data.items():
        r3_pnls[trader][product] = calc_pnl(trader, trades)

for trader in r4_traders:
    data = {product: round_4[round_4["symbol"] == product] for product in r4_products}
    for product, trades in data.items():
        r4_pnls[trader][product] = calc_pnl(trader, trades)

{'Vinnie': {'AMETHYSTS': -680634.0, 'STARFRUIT': 3917522.0}, 'Remy': {'AMETHYSTS': -243626.0, 'STARFRUIT': 2504569.0}, 'Ruby': {'AMETHYSTS': 2887948.0, 'STARFRUIT': 829031.0}, 'Rhianna': {'AMETHYSTS': -356930.0, 'STARFRUIT': 749043.0}, 'Valentina': {'AMETHYSTS': 150418.0, 'STARFRUIT': -7315171.0}, 'Vladimir': {'AMETHYSTS': -873652.0, 'STARFRUIT': 499378.0}, 'Adam': {'AMETHYSTS': 38032.0, 'STARFRUIT': -336004.0}, 'Amelia': {'AMETHYSTS': -921556.0, 'STARFRUIT': -848368.0}}
